# Centrality Tutorial

This notebook covers some of the centrality algorithms implemented in NetworKit. [Centrality](http://en.wikipedia.org/wiki/Centrality) measures the relative importance of a node within a graph. Code for centrality analysis in NetworKit is grouped into the [centrality](https://networkit.github.io/dev-docs/python_api/centrality.html) module.

As always, the first step is to start by importing NetworKit.

In [1]:
import networkit as nk

Every algorithm within the `centrality` module in NetworKit has a `run()` function that must be called in order for the algorithm to be executed after initialization.

## Betweeness Centrality

Betweenness centrality measures the extent to which a vertex lies on shortest paths between other vertices. Vertices with high betweenness may have considerable influence within a network.

### Betweenness

The fastest algorithm for the exact computation of the betweenness centrality is Brandes' algorithm, and it is implemented in the [Betweenness](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=betweenness#networkit.centrality.Betweenness) class. The constructor [Betweenness(G, normalized=False, computeEdgeCentrality=False)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=betweenness#networkit.centrality.Betweenness) expects a mandatory [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph), and two optional parameters ` normalized` and `computeEdgeCentrality`. If `normalized` is set to true the centrality scores will be normalized. The parameter `computeEdgeCentrality` should be set to true if edge betweeness should be computed as well. 

We start by reading a graph, and then inititalising the algorithm with the parameters we want. Assuming we shall use the default parameters, i.e., centrality scores should not be normalized and edge betweeness should not be computed, we only need to pass a graph to the method.

In [ ]:
# Read graph
G = nk.readGraph("../input/power.graph", nk.Format.METIS)
# Initalize algorithm
btwn = nk.centrality.Betweenness(G)

In [ ]:
# Run 
btwn.run()

In [ ]:
# The 10 most central nodes according to betweenness are then
btwn.ranking()[:10]

### ApproxBetweenness

Since exact calculation of betweenness scores is often out of reach, NetworKit provides an approximation algorithm based on path sampling. This functionality is provided by the ApproxBetweenness class, [(G, epsilon=0.01, delta=0.1, universalConstant=1.0)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=approx#networkit.centrality.ApproxBetweenness). It expects a mandatory undirected [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph). Here we estimate betweenness centrality in `PGPgiantcompo`, with a guarantee that the error is no larger than an additive constant `epsilon` with a probability of at least 1 - `delta`. The `universalConstant` is used to compute the sample size.

In [ ]:
# Read graph
G = nk.readGraph("../input/PGPgiantcompo.graph", nk.Format.METIS)

In [ ]:
ab = nk.centrality.ApproxBetweenness(G, epsilon=0.1)
ab.run()

In [ ]:
# The 10 most central nodes according to betweenness are then
ab.ranking()[:10]

### EstimateBetweenness

The [EstimateBetweenness](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=estimate#networkit.centrality.EstimateBetweenness) class estimates the betweenness centrality according to the algorthm described in `Sanders, Geisberger, Schultes: Better Approximation of Betweenness Centrality`.
Despite the algorithm performing well in practice, no guarantee can be given. If a theoritical guarantee is required, use the [ApproxBetweenness](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=approx#networkit.centrality.ApproxBetweenness) algorithm.

The constructor `EstimateBetweenness(Graph G, count nSamples, bool normalized=False, bool parallel_flag=False)` expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) and the number of samples as mandatory parameters. Further, the centrality values can be optionally normalized by setting `normalized` to `True`; by default the centrality values are not normalized. This algorithm can be run in parallel by setting the `parallel_flag`  to true. Running in parallel, however, comes with an additional cost in memory of z + 3z + t. 

In [ ]:
# Read a graph
G = nk.readGraph("../input/PGPgiantcompo.graph", nk.Format.METIS)

In [ ]:
# Initialize algorithm
est = nk.centrality.EstimateBetweenness(G, 50, True, False)
est.run()

In [ ]:
#The 10 most central nodes according to betweenness are then
est.ranking()[:10]

### KadabraBetweenness

[KadabraBetweenness](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=kadabra#networkit.centrality.KadabraBetweenness) is an ADaptive Algorithm for Betweennes via Random Approximation presented by [Borassi M., and Natale E.](https://arxiv.org/abs/1604.08553).
NetworKit provides variants of the algorithm that either reduce this memory consumption to O(1) or ensure that deterministic results are obtained. For more details about the implementation, see this [paper](https://arxiv.org/abs/1903.09422) by `Van der Grinten A., Angriman E., and Meyerhenke H. (2019)`.

The constructor [KadabraBetweennes(G, err=0.01, delta=0.1, deterministic=False, k=0, unionSample=0, startFactor=100)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=kadabra#networkit.centrality.KadabraBetweenness) only expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) as a mandatory parameter. `err` is the maximum additive error guaranteeded when approximating the betweenness centrality of all nodes, while `delta` is the probability that the values of the betweenness centrality are within the error guarantee. The parameter `k` expresses the number of top k-nodes to be computed; if set to zero the approximate betweenness centrality of all nodes will be computed. `unionSample` and `startFactor` are algorithm parameters that are automatically chosen.

Hence, computing the KadabraBetweenness for all nodes with a maximum additive error of 0.05, and a probabilty of 0.8 that the values of the betweenness centrality are within that range can be done as follows:

In [ ]:
# Read a graph
G = nk.readGraph("../input/PGPgiantcompo.graph", nk.Format.METIS)

In [ ]:
# Initialize algorithm
kadabra = nk.centrality.KadabraBetweenness(G, 0.05, 0.8)
kadabra.run()

In [ ]:
#The 10 most central nodes according to betweenness are then
kadabra.ranking()[:10]

##  Closeness Centrality

Closeness centrality indicates how close a node is to all other nodes in the network. It is calculated as inverse of the average of the shortest path length from the node to every other node in the network.

### Closeness

The NetworKit Closeness class computes the exact closeness centrality of all the nodes of a graph. The constructor [Closeness(G, normalized=True, variant=networkit.centrality.ClosenessVariant.STANDARD)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=closeness#networkit.centrality.Closeness) expects a mandatory [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph), and two optional parameters, `normalized` and `variant`. The centrality values can be optionally normalized for unweighted graphs by setting `normalized` to True; by default the centrality values are normalized. The parameter`variant` dictates the closeness variant to be used when computing the closeness. Set `variant` to `networkit.centrality.ClosenessVariant.STANDARD` to use the standard definition of closeness  which is defined for connected graphs only, or to `networkit.centrality.ClosenessVariant.GENERALIZED` to use the generalized definition of closeness that is also defined for non-connected graphs. 

As the graph we will use is weighted the values should not be normalized. So, to compute the closeness of a graph using the generalised closeness variant because our graph is disconnected we can do the following:

In [ ]:
# Read a graph
G = nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT)

In [ ]:
# Initialize algorithm
close = nk.centrality.Closeness(G, False, nk.centrality.ClosenessVariant.GENERALIZED)
close.run()

In [ ]:
#The 10 most central nodes according to betweenness are then
close.ranking() [:10]

### ApproxCloseness

Since exact calculation of closeness scores is often out of reach, NetworKit provides an approximation closeness centrality according to the algorithm described in '`Cohen et al., Computing Classic Closeness Centrality, at Scale`. 

The constructor expects a mandatory connected [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) among other optional values:
[ApproxCloseness(G, nSamples, epsilon=0.1, normalized=False, type=OUTBOUND)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=approx#networkit.centrality.ApproxCloseness). `nSamples` dictates the number of samples that should be used when computing the approximate closeness; this value depends on the size of graph and the more samples that are used, the better the results are likely to be. `epsilon` can be any value between 0 and $\infty$-1   and is used for the error guarantee. The centrality values can be optionally normalized by setting `normalized` to True; by default the centrality values are not normalized. If G is undirected, `type` can be ignored. Otherwise use `0` for inbound centrality, `1` for outbound centrality and `2` for the sum of both.

In [ ]:
# Read a graph
G = nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT)

In [ ]:
ac = nk.centrality.ApproxCloseness(G, 100)
ac.run()

In [ ]:
# 10 most central nodes according to closeness are 
ac.ranking()[:10]

### HarmonicCloseness

Harmonic centrality is a variant of closeness centrality. It deals with the problem of unconnected graphs.
The constructor, [HarmonicCloseness(G, normalized=True)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=harmon#networkit.centrality.HarmonicCloseness), expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) and an optional parameter `normalized` that is set to true by default. If `normalized` is set to true, centrality scores are normalized to into an interval between 0 and 1. 

Computing the harmonic closeness of a graph, and then normalizing the scores can be done as follows:

In [ ]:
# Read a graph
G = nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT)

In [ ]:
harmonic = nk.centrality.HarmonicCloseness(G, True)
harmonic.run()

In [ ]:
# 10 most central nodes according to closeness are
harmonic.ranking()[:10]

### TopHarmonicCloseness

The [TopHarmonicCloseness(G, k=1, useBFSbound=True)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=top#networkit.centrality.TopHarmonicCloseness) algorithm finds the exact top k nodes with highest harmonic closeness centrality. It is faster than computing the harmonic closeness for all nodes. The implementation is based on [Computing Top-k Centrality Faster in Unweighted Graphs, Bergamini et al., ALENEX16](https://arxiv.org/abs/1710.01143). The parameter `k` expresses the number of nodes with the highest centrality scores that the algorithm must find. The algorithms are based on two heuristics. We recommend to use `useBFSbound = False` for complex networks (or networks with small diameter) and `useBFSbound = True` for street networks (or networks with large diameters). 

In [ ]:
# Read a graph
G = nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT)

In [ ]:
# Initialize algorithm
topHarmClose = nk.centrality.TopHarmonicCloseness(G, 10, useNBbound=False)
topHarmClose.run()

Unlike all the other algorithms we have seen before, `TopHarmonicCloseness` does not have a `ranking` method which stores a sorted list of all centrality scores. Instead, we have the [topkNodesList(includeTrail=False)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=topknodes#networkit.centrality.TopHarmonicCloseness.topkNodesList) method. By calling this method, we can print the the top `k` central nodes. If `includeTrail` is true the closeness centrality of some nodes below the top-k could be equal to the k-th closeness will also be printed. Note, that the resulting vector may be longet than `k`.

In [ ]:
# k most central nodes according to closeness are
topHarmClose.topkNodesList()

Alternatively, the [topKScores(includeTrail=False)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=topkscores#networkit.centrality.TopCloseness.topkScoresList) method may be used.

In [ ]:
topHarmClose.topkScoresList()

### TopCloseness

The [TopCloseness(G, k=1, first_heu=True, sec_heu=True)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=topcloseness#networkit.centrality.TopCloseness) algorithm finds the exact top k nodes with highest harmonic closeness centrality. It is faster than computing the harmonic closeness for all nodes. The implementation is based on [Computing Top-k Centrality Faster in Unweighted Graphs, Bergamini et al., ALENEX16](https://arxiv.org/abs/1710.01143). The algorithm is based on two independent heuristics described in the above paper. The parameter `k` expresses the number of nodes with the highest centrality scores that have to be found by the algorithm. If `first_heu` is true,  the neighborhood-based lower bound is computed and nodes are sorted according to it. If false, nodes are simply sorted by degree. If `sec_heu` is true, the BFSbound is re-computed at each iteration. If false, BFScut is used.



In [ ]:
# Read a graph
G = nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT)

In [ ]:
# Initialize algorithm
topClose = nk.centrality.TopCloseness(G, 10, first_heu=True)
topClose.run()

In [ ]:
# k most central nodes according to closeness are
topClose.topkNodesList()

## Degree Centrality

Degree is a centrality measure that counts how many neighbors a node has.

### Degree Centrality

Degree centrality is a centrality measure that counts how many neighbors a node has. Degree centrality can be computed in NetworKit using the [DegreeCentrality(G, normalized=False, outDeg=True, ignoreSelfLoops=True)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=degreecentrality#networkit.centrality.DegreeCentrality) class. It expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph). Set `normalized` to True if the centrality scored should be normalized to values between 0 and 1. If the network is directed, we have two versions of the measure: in-degree is the number of incoming links; out-degree is the number of outgoing links. By default, the out-degree is used. 

For an undirected graph, using the default parameters, the degree centrality can be computed as follows:

In [ ]:
# Read graph 
G = nk.readGraph("../input/wiki-Vote.txt", nk.Format.SNAP)

In [ ]:
# Initialize algorithm
deg = nk.centrality.DegreeCentrality(G)
deg.run()

In [ ]:
# 10 most central nodes according to degree centrality are
deg.ranking()[:10]

## Eigenvector Centrality and PageRank

Eigenvector centrality measures the influence of a node in a network by assinging relative scores to each node in a network.

### Eigenvector Centrality and PageRank

Eigenvector centrality and its variant PageRank assign relative importance to nodes according to their connections, incorporating the idea that edges to high-scoring nodes contribute more. PageRank is a version of eigenvector centrality which introduces a damping factor, modeling a random web surfer which at some point stops following links and jumps to a random page. In PageRank theory, centrality is understood as the probability of such a web surfer to arrive on a certain page. Our implementation of both measures is based on parallel power iteration, a relatively simple eigensolver.

We demonstrate it here on the small Karate club graph. 

In [2]:
# Read graph 
K = nk.readGraph("../input/karate.graph", nk.Format.METIS)

In [ ]:
# Eigenvector centrality
ec = nk.centrality.EigenvectorCentrality(K)
ec.run()
ec.ranking()[:10] # the 10 most central nodes

In [3]:
# PageRank
pr = nk.centrality.PageRank(K, damp=0.85, tol=1e-9)
pr.run()
pr.ranking()[:10] # the 10 most central nodes

[(33, 0.10091918107710363),
 (0, 0.09699728673391753),
 (32, 0.07169322502169574),
 (2, 0.05707850945632192),
 (1, 0.05287692444772233),
 (31, 0.03715808671239322),
 (3, 0.035859858105808544),
 (23, 0.031522514279355436),
 (8, 0.02976605594632442),
 (13, 0.029536456269206687)]

By default, PageRank uses the L2 norm as convergence criterion. Alternatively, one can also use the L1 norm.
Additionally, one can also set the maximum amount of iterations.

In [ ]:
# PageRank using L1 norm, and a 100 maximum iterations
pr = nk.centrality.PageRank(K, damp=0.85, tol=1e-9)
pr.norm = nk.centrality.Norm.L1_NORM
pr.maxIterations = 100
pr.run()
pr.ranking()[:10] # the 10 most central nodes

### Katz Centrality

Katz centrality computes the relative influence of a node within a network by measuring the number of the immediate neighbors, and also all other nodes in the network that connect to the node through these immediate neighbors. Connections made with distant neighbors are, however, penalized by an attenuation factor $\alpha$. Each path or connection between a pair of nodes is assigned a weight determined by $\alpha$ and the distance between nodes as $\alpha ^d$.

The [KatzCentrality(G, alpha=5e-4, beta=0.1, tol=1e-8)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=katz#networkit.centrality.KatzCentrality) constructor expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) as a mandatory parameter. The parameter `alpha` is the damping of the matrix vector result, while `beta` is a constant value added to the centrality of each vertex. The parameter `tol` dictates the tolerance for convergence. Computing the Katz centrality in NetworKit can be like below.  

In [ ]:
G = nk.readGraph("../input/karate.graph", nk.Format.METIS)

In [ ]:
# Initialize algorithm
katz = nk.centrality.KatzCentrality(G, 1e-3)
katz.run()

In [ ]:
# 10 most central nodes
katz.ranking()[:10]

## Others

### Spanning Edge Centrality

The spanning centrality of an edge `e` in an undirected graph is the fraction of the spanning trees of the graph that contain the edge `e`. The [SpanningEdgeCentrality(const Graph& G, double tol = 0.1)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=spanning#networkit.centrality.SpanningEdgeCentrality) constructor expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) as a mandatory parameter. The `tol` parameter determines the tolerance used for approximation: with probability at least 1-1/n, the approximated scores are within a factor 1+`tol` from the exact scores. 

Computing the SpanningEdge Centrality in NetworKit is then demonstrated below.

In [ ]:
# Read graph 
G = nk.readGraph("../input/karate.graph", nk.Format.METIS)

In [ ]:
# Initialize algorithm
G.indexEdges()
span = nk.centrality.SpanningEdgeCentrality(G, 1e-6)
span.run()

Using the [scores()](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=scores#networkit.centrality.SpanningEdgeCentrality.scores) method we can get a vector containing the SEC score for each edge in the graph.

In [ ]:
span.scores()[:10]

### Approximate Spanning Edge Centrality

An algorithm from [Hayashi et al.](https://www.ijcai.org/Proceedings/16/Papers/525.pdf) based on sampling of spanning trees uniformly at random approximates the spanning edge centrality of each edge of a graph up to a maximum absolute error. `ApproxSpanningEdge(G, eps=0.1)` expects as input an undirected graph, and the maximum absolute error `eps`. The algorithm requires the edges of the graph to be indexed.

In [ ]:
# Read a graph
G = nk.graphtools.toUndirected(nk.readGraph("../input/foodweb-baydry.konect", nk.Format.KONECT))
G.indexEdges()
apx_span = nk.centrality.ApproxSpanningEdge(G, 0.05)
_=apx_span.run()

In [ ]:
apx_span.scores()[:10]

### Local ClusteringCoefficient

The local clustering coefficient of a vertex in a graph quantifies how close its neighbours are to being a clique (complete graph).

The [LocalClusteringCoefficient(G, turbo=False)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=coeff#networkit.centrality.LocalClusteringCoefficient) constructor expects a [networkit.Graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) and a Boolean parameter `turbo` which is by default initialized to false. `turbo` activates a (sequential, but fast) pre-processing step using ideas from [this paper](https://dl.acm.org/citation.cfm?id=2790175). This reduces the running time significantly for most graphs. However, the turbo mode needs O(m) additional memory. The turbo mode is particularly effective for graphs with nodes of very high degree and a very skewed degree distribution.

We demonstrate the use using the same graph as before, 

In [ ]:
# Read graph 
G = nk.readGraph("../input/karate.graph", nk.Format.METIS)

In [ ]:
# Initialize algorithm
lcc = nk.centrality.LocalClusteringCoefficient(G)
lcc.run()

In [ ]:
lcc.ranking()[:10]